## Code to transform the dataset

In [82]:
!pip install transformers -q
!pip install torch -q

In [388]:
import os
from tqdm import tqdm
import pandas as pd
import transformers, torch
from PIL import Image, ImageOps
from torchvision import transforms
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import importlib
import os
from tqdm import tqdm
import numpy as np
import shutil

In [402]:
import model_functions, utils, training, datasets
def update():# if you change our files
    import model_functions, utils, training, datasets
    for lib in [model_functions, utils, training, datasets]:
        importlib.reload(lib)# issues with not updating
update()

In [403]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available(): # For apple silicon
    device = 'mps'
print("Using device:", device)

Using device: cuda


In [ ]:
# ./HM_DATA_PATH/images/..., ./HM_DATA_PATH/articles.csv, etc
HM_DATA_PATH = "./data/"

# path where new dataset will be saved to
HM_DATA_PATH_NEW = "./dataset/"

SRC_IMAGE_DIR = HM_DATA_PATH + "images"
DEST_IMAGE_DIR = HM_DATA_PATH_NEW + "images"

DEST_CSV = HM_DATA_PATH_NEW + "articles.csv"

cwd = os.getcwd()
print(f"Current working directory: {cwd}")
if not os.path.exists(DEST_IMAGE_DIR):
    print(f"Directory does not exist: {DEST_IMAGE_DIR}")

In [404]:
def get_number_image_ids():
    """Loop trough all files in the folder and get image_ids from name"""
    file_names =[]
    for root, _, files in os.walk(DEST_IMAGE_DIR):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):# article id
                if len(file[1:-4])==9:
                    file_names.append(int(file[1:-4]))
    return file_names

def create_folders():#markus
    if not os.path.exists(DEST_IMAGE_DIR):
        os.makedirs(DEST_IMAGE_DIR)


def copy_csv_json_files(all_files=False):#markus
    # only copy articles.csv
    shutil.copy2(HM_DATA_PATH + "articles.csv", HM_DATA_PATH_NEW + "articles.csv")

    # only copy over .csv and .json
    if all_files:
        for file in os.listdir(HM_DATA_PATH):
            if file.endswith(".csv") or file.endswith(".json"):
                src = HM_DATA_PATH + file
                dest = HM_DATA_PATH_NEW + file

                os.makedirs(os.path.dirname(dest), exist_ok=True)
                shutil.copy2(src, dest)


def images_processing(): #markus
    # go through all images
    for root, _, files in os.walk(SRC_IMAGE_DIR):
        for file in tqdm(files):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                try:
                    img_path = os.path.join(root, file)
                    img = Image.open(img_path)

                    # when this works it's good. Otherwise it's bad.

                    # save rgb of all pixels of border of image
                    # np_image = np.array(img)

                    # top_border = np_image[1, 1:-1, :]
                    # bottom_border = np_image[-2, 1:-1, :]
                    # left_border = np_image[1:-1, 1, :]
                    # right_border = np_image[1:-1, -2, :]

                    # border_pixels = np.concatenate([top_border, bottom_border, left_border, right_border], axis=0)

                    # def get_brightest_or_closest_to_white(border_pixels):
                    #     def distance_to_white(pixel):
                    #         r, g, b = pixel
                    #         return np.sqrt((255 - r) ** 2 + (255 - g) ** 2 + (255 - b) ** 2)
                    
                    #     # Find the pixel with the highest brightness (or closest to white)
                    #     brightest_pixel = min(border_pixels, key=distance_to_white)
                    #     return tuple(brightest_pixel)

                    # Choose the brightest or closest to white color for padding
                    # padding_color = get_brightest_or_closest_to_white(border_pixels)


                    # just pick RGB of background manually and assign to every image.
                    padding_color = (236, 235, 233)
                    r,g,b = padding_color

                    # rezise and add padding
                    square_width_height = 224 # transformer take 224x224 image res
                    img.thumbnail((square_width_height, square_width_height), Image.LANCZOS)
                    padding = (square_width_height - img.size[0], square_width_height - img.size[1])
                    img = ImageOps.expand(img, (padding[0]//2, padding[1]//2, (padding[0]+1)//2, (padding[1]+1)//2), fill=(r,g,b))

                    # save new image
                    output_path = os.path.join(DEST_IMAGE_DIR, file)
                    os.makedirs(os.path.dirname(output_path), exist_ok=True)
                    img.save(output_path)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
                    
def downscale_folders():#markus
    create_folders()
    copy_csv_json_files()
    images_processing()
#downscale_folders()

Current working directory: /home/jupyter/DD2430_Project/src


**Load all data into tensors**

In [405]:
image_ids = get_number_image_ids()
df = pd.read_csv(DEST_CSV)
article_id = df['article_id'].values
print(len(article_id),len(image_ids))
print(len(article_id)-len(image_ids), 'are missing')
common_ids = set(article_id).intersection(image_ids)
different_ids = set(article_id).difference(image_ids)
print(len(common_ids), len(different_ids))

filtered_df = df[~df['article_id'].isin(different_ids)]
print(len(filtered_df))
filtered_df.to_csv(HM_DATA_PATH_NEW+'articles_filtered.csv', index=True) 

105542 105099
443 are missing
105099 443
105099


In [ ]:
model = transformers.CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = transformers.CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
processor.feature_extractor.do_rescale = False # make sure image values: False=> [0-1] and True=> [0,255]
clip = {'m':model, 'p':processor}

In [ ]:
import model_functions
def get_data_as_tensors(path, clip, limit =10): # takes 20min for all
    """Loops trough all folder and returns a 0.5 GB tensor of emb for all 100k images,  and a label of input_ids"""
    labels, images, embeds, count = torch.zeros(limit, dtype=torch.int32), [],torch.zeros((limit,512), dtype=torch.float32), 0
    transform = transforms.ToTensor()
    with tqdm(total=limit, desc="Loading", unit="Images") as pbar:
        for root, _, files in os.walk(path):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    article_id_im = file[1:-4]
                    if len(article_id_im)==9 and count<limit: # article id 
                        img_path = os.path.join(root, file)
                        image = transform(Image.open(img_path)).to(device)
                        # images.append(image)
                        labels[count]=int(article_id_im)
                        with torch.no_grad():
                            image_emb, _ = model_functions.get_image_emb(model, processor, image, normalize=True)
                        embeds[count]=(image_emb.squeeze()).cpu()
                        count +=1
                        del image
                        del image_emb
                        del _
                        torch.cuda.empty_cache() 
                        pbar.update(1)
    return embeds, labels #[X, 3, 224, 224]
    # full image tensor is [100000,3,224,224] thus 55 GB not possible to fit in ram
    #2048 Byte per emb thus 0.2GB for full ([X, 512])
    
embeds, labels = get_data_as_tensors(DEST_IMAGE_DIR, clip, len(common_ids))
print(embeds.shape, labels.shape)
def bytes_tensor(tensor):
    size_in_bytes = tensor.element_size() * tensor.numel()
    print(f"Size of PyTorch tensor: {size_in_bytes} bytes")

torch.save(embeds, HM_DATA_PATH_NEW+'embedds.pth')
torch.save(labels, HM_DATA_PATH_NEW+'labels.pth')

bytes_tensor(embeds)

Loading: 100%|██████████| 105099/105099 [19:39<00:00, 89.12Images/s]


torch.Size([105099, 512]) torch.Size([105099])
Size of PyTorch tensor: 215242752 bytes


In [412]:
df = pd.read_csv(HM_DATA_PATH_NEW+'articles_filtered.csv')
embs = torch.load(HM_DATA_PATH_NEW+'embedds.pth', weights_only=True)
labs = torch.load(HM_DATA_PATH_NEW+'labels.pth', weights_only=True).tolist()
print(len(labs))

105099
